In [2]:
import pandas as pd
import numpy as np

In [1]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
data_dictionary = pd.read_csv('data_dictionary.csv')


NameError: name 'pd' is not defined